# [Atividade Computacional 3](http://mtm.ufsc.br/~douglas/2025.1/MTM410024_MTM3523/Atividade3.pdf)

Em muitas funções, utilizei a macro `@views`, que faz com que as operações não
criem cópias de vetores para os cálculos, otimizando a performance, conforme a
sessão [Performance tips](https://docs.julialang.org/en/v1/manual/performance-tips/index.html#man-performance-views) do Manual do `julia`.

In [1]:
using LinearAlgebra

## Questão 1

Implementar Gram-Schimidt

In [2]:
"""
  GramSchimidt(A)

  Aplica o processo de Gram-Schimidt as colunas de A, retornando uma matriz Q
  com as colunas ortogonais.
"""
function GramSchimidt(A::Matrix)
  m, n = size(A)
  # Converte para Matrix{Float64} para evitar erros
  Q = convert(Matrix{Float64},copy(A))
  Q[:,1] = A[:,1]/norm(A[:,1])
  for i = 2:n
    projecao = zeros(m)
    for k=1:i-1
      projecao += Q[:,k]'A[:,i]*Q[:,k]
    end
    qi = A[:,i] - projecao
    Q[:,i] = qi/norm(qi)
  end
  return Q
end

GramSchimidt

## Questão 2

Aplicar o processo aos vetores $x_1, x_2, x_3$

In [3]:
x1 = [1, 0, 0, -1]
x2 = [1, 2, 0, -1]
x3 = [3, 1, 1, -1];

In [4]:
Q = GramSchimidt([x1 x2 x3])

4×3 Matrix{Float64}:
  0.707107   1.11022e-16   0.57735
  0.0        1.0          -2.56395e-16
  0.0        0.0           0.57735
 -0.707107  -1.11022e-16   0.57735

## Questão 3

Verificando que são ortogonais.

De fato são, pois $10^{-16}\approx 0$

In [5]:
Q'Q

3×3 Matrix{Float64}:
 1.0           1.57009e-16   2.50235e-16
 1.57009e-16   1.0          -2.56395e-16
 2.50235e-16  -2.56395e-16   1.0

In [ ]:
# Usando a função isapprox(x,y)
Q'Q ≈ I(3)

true

## Questão 4

Aplicar ortogonalização à vetores "quase" LD.

In [6]:
ϵ = 1.0e-5
x1 = [1, ϵ, 0, 0]
x2 = [1, 0, ϵ, 0]
x3 = [1, 0, 0, ϵ]

Q = GramSchimidt([x1 x2 x3])

4×3 Matrix{Float64}:
 1.0      7.07107e-6   4.08248e-6
 1.0e-5  -0.707107    -0.408248
 0.0      0.707107    -0.408248
 0.0      0.0          0.816497

In [ ]:
X = Q'Q

3×3 Matrix{Float64}:
 1.0           5.8577e-13   6.76185e-13
 5.8577e-13    1.0         -4.78279e-8
 6.76185e-13  -4.78279e-8   1.0

O valor de $ u2^\top u3 $ está em $X_{2,3}$

In [ ]:
X[2,3]

-4.7827910485723726e-8

Veja que não está suficientemente próximo a zero.

In [ ]:
isapprox(X[2,3], 0, atol=1.0e-15)

false

## Questão 5

In [7]:
function modGramSchimidt(A::Matrix)
  V = convert(Matrix{Float64}, copy(A))
  m, n = size(V)
  Q = zeros(m,n)
  R = zeros(n,n)
  for i = 1:n
    R[i,i] = norm(V[:,i])
    Q[:,i] = V[:,i]/R[i,i]
    for j=i+1:n
      R[i,j] = Q[:,i]'V[:,j]
      V[:,j] = V[:,j] - R[i,j]*Q[:,i]
    end
  end
  return Q, R
end

modGramSchimidt (generic function with 1 method)

In [8]:
Q, R = modGramSchimidt([x1 x2 x3]);

In [9]:
X = Q'Q

3×3 Matrix{Float64}:
 1.0           5.8577e-13    3.38194e-13
 5.8577e-13    1.0          -3.96518e-17
 3.38194e-13  -3.96518e-17   1.0

O valor de $ u2^\top u3 $ está em $X_{2,3}$, e dessa vez, está próximo de zero.

In [10]:
X[2,3]

-3.9651816033099527e-17

In [ ]:
isapprox(X[2,3], 0, atol=1.0e-15)

true

In [11]:
Q*R ≈ [x1 x2 x3]

true